In [1]:
import gc
from sklearn.decomposition import NMF

gc.enable()

In [2]:
import pickle

with open("Data/df.pkl", 'rb') as picklefile:
    df = pickle.load(picklefile)

Let's do our analysis on stemmed words so that same topic is not split (e.g. "word" and "words" should belong to the same topic):

In [3]:
# import nltk
# from textblob import TextBlob
# stemmer = nltk.stem.porter.PorterStemmer()

# def stem_getter(text):
#     return " ".join([stemmer.stem(word) for word in TextBlob(text).words])

# df.raw_text = df.raw_text.map(stem_getter)

This time around, let's remove words in all caps: they are used to indicate character lines. Using them will just create topics identifying major characters of a show/movie which is not helpful. Let's also remove non-letter characters along the way:

In [4]:
import re

In [5]:
def cap_remover(text):
    text = re.sub(r'[A-Z]+(?![a-z])', '', text)
    text = re.sub(r'[\d]+', '', text)
    text = re.sub(r' +', ' ', text)
    return re.sub(r"[^\w' ]", '', text)

In [6]:
df.raw_text = df.raw_text.map(cap_remover)

In [7]:
%pylab inline
import numpy 
import matplotlib.pyplot as plt
import sklearn
# Import all of the scikit learn stuff
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


I have also used a large list of keywords from [here](http://www.ranks.nl/stopwords) and supplemented it with Star Trek specific terms discovered in the initial LDA model so that they are not used in topic analysis:

In [8]:
with open('Data/stopwords.txt') as f:
     content = (f.read()).split()
stoplist = sorted(list(set(content)))

In [9]:
from helper import *
import warnings
warnings.filterwarnings('ignore')

In [10]:
import nltk
# nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/aleksod/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stopwords = stoplist #set(stopwords.words('english'))
punctuation = set(string.punctuation) 
lemmatize = WordNetLemmatizer()

def cleaning(article):
    one = " ".join([i for i in article.lower().split() if i not in stopwords])
    two = "".join(i for i in one if i not in punctuation)
    three = " ".join(lemmatize.lemmatize(i) for i in two.split())
    return three

In [12]:
# df2 = df.drop(["_id","end","series","start","url","airdate"], axis=1)

In [13]:
text = df.raw_text.map(cleaning) #['raw_text']
text_list = [i.split() for i in text]
len(text_list)

678

In [14]:
# docs = df.raw_text
count_vectorizer = CountVectorizer(stop_words = stoplist, ngram_range=(1, 3))
dtm = count_vectorizer.fit_transform(text) #docs) 

In [15]:
df = None
df2 = None
docs = None
del df
del df2
del docs
gc.collect()

7

In [16]:
# gc.get_objects()

Let's see what 10 topics could be:

In [17]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_jobs=-1)
lda.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_jobs=-1, n_topics=10, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [18]:
import pyLDAvis, pyLDAvis.sklearn
from IPython.display import display


# Setup to run in Jupyter notebook
pyLDAvis.enable_notebook()

# Create the visualization
vis = pyLDAvis.sklearn.prepare(lda, dtm, count_vectorizer)

# # Export as a standalone HTML web page
# pyLDAvis.save_html(vis, 'lda.html')

# Let's view it!
display(vis)

/home/aleksod/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      89.195681        1       1 -0.197259  0.009722
4       3.417270        1       2  0.040558  0.116923
0       1.678142        1       3  0.027564 -0.023059
6       1.653640        1       4  0.025736 -0.021923
2       1.206135        1       5  0.022629 -0.017502
9       0.812284        1       6  0.022928 -0.019702
8       0.684541        1       7  0.022414 -0.021250
1       0.493545        1       8  0.015706 -0.012243
3       0.436060        1       9  0.010146 -0.005809
7       0.422703        1      10  0.009577 -0.005156, topic_info=        Category          Freq                 Term         Total  loglift  \
term                                                                         
1526014  Default   3815.000000               mister   3815.000000  30.0000   
262772   Default   5896.000000               bridge   5896.000000  29.0000   
402269   Default    570.000000                 code    570.000000  28.0000   
1844600  Default    768.000000              pulaski    768.000000  27.0000   
2166685  Default   8726.000000                  sir   8726.000000  26.0000   
2076016  Default    843.000000                 send    843.000000  25.0000   
1472677  Default   1665.000000               matter   1665.000000  24.0000   
492012   Default   3182.000000              control   3182.000000  23.0000   
1755664  Default   2907.000000               planet   2907.000000  22.0000   
2502750  Default   2098.000000          transporter   2098.000000  21.0000   
2036031  Default    235.000000                sarek    235.000000  20.0000   
179913   Default   8966.000000                 beat   8966.000000  19.0000   
2677906  Default  10546.000000                 will  10546.000000  18.0000   
2052622  Default    422.000000                scott    422.000000  17.0000   
1171076  Default   2270.000000                human   2270.000000  16.0000   
1476957  Default    234.000000                   mc    234.000000  15.0000   
1441730  Default   1153.000000                 main   1153.000000  14.0000   
2699458  Default   1726.000000                woman   1726.000000  13.0000   
2111441  Default  10405.000000                 ship  10405.000000  12.0000   
2595603  Default    711.000000               viewer    711.000000  11.0000   
1373051  Default   2527.000000           lieutenant   2527.000000  10.0000   
715489   Default   1265.000000                earth   1265.000000   9.0000   
1954985  Default   1728.000000               report   1728.000000   8.0000   
2504259  Default    641.000000     transporter room    641.000000   7.0000   
2612209  Default   1498.000000               vulcan   1498.000000   6.0000   
1662768  Default   2068.000000                order   2068.000000   5.0000   
1417093  Default    138.000000                 lore    138.000000   4.0000   
2263330  Default    383.000000             starbase    383.000000   3.0000   
174984   Default   2079.000000                 beam   2079.000000   2.0000   
1660588  Default   1239.000000                orbit   1239.000000   1.0000   
...          ...           ...                  ...           ...      ...   
548848   Topic10      0.223153        crystal shape      0.471581   4.7180   
571714   Topic10      0.222900    datalore datalore      0.473165   4.7135   
1529085  Topic10      0.222655         mister logan      0.473992   4.7107   
75146    Topic10      0.277011         android form      0.597717   4.6972   
2651882  Topic10      0.546376  weapon control area      1.301869   4.5980   
2243143  Topic10      0.385061         sphere forty      0.922295   4.5928   
116126   Topic10      3.315452              arsenal     13.987119   4.0267   
1959280  Topic10      0.871312     reptilian bridge      2.844393   4.2832   
603133   Topic10      1.457169          degras ship      7.911839   3.7744   
603104   Topic10      1.765530   

In [21]:
lda.components_.shape

(10, 2748255)

In [ ]:
with open("Data/ldamodel1.pkl", 'wb') as picklefile:
    pickle.dump(ldamodel, picklefile)

In [ ]:
print(ldamodel.print_topics(num_topics=2, num_words=4))

In [ ]:
for i in ldamodel.print_topics(): 
    for j in i: print(j)

Brilliant! My topics are based on characters that are in the respective series! Not a very useful information, but it LDA works pretty well. Let's see if we can visuzlize it.

In [ ]:
ldamodel.save('topic.model')

In [ ]:
from gensim.models import LdaModel
loading = LdaModel.load('topic.model')

In [ ]:
print(loading.print_topics(num_topics=2, num_words=4))

In [ ]:
def pre_new(doc):
    one = cleaning(doc).split()
    two = dictionary.doc2bow(one)
    return two

In [ ]:
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()

In [ ]:
d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')

In [ ]:
data = pyLDAvis.gensim.prepare(lda, c, d)
data

In [ ]:
pyLDAvis.save_html(data,'vis1.html')

This is a pretty good separation. Unfortunately, it is based mostly on very specific TV show / movies terms. Therefore, I will need to remove those if I want better results.